# Extract rent/sell prices per m2 and number of Airbnb offers data sets

In the current notebook, we are extracting the data available online into a computer readable shape (data frame). We also need to group data from different years and label it correctly according to the dates.

* The source of rent and sell prices per m2 per district in Madrid is [idealista.com](https://www.idealista.com/informes-precio-vivienda) for the years 2007-2017
* The source of Airbnb listings is [insideairbnb.com](http://insideairbnb.com/get-the-data.html) for several months in 2015, 2017 and 2018. I will need a reconstruction of the trend to extrapolate data to missing dates.

In [2]:
import datetime
import pandas as pd
import numpy as np
from tabula import read_pdf
import warnings
warnings.filterwarnings('ignore')

In [2]:
upper = ['Barajas', 'Carabanchel',
       'Centro', 'Chamartín', 'Chamberí', 'Ciudad Lineal', 'Fuencarral',
       'Hortaleza', 'Latina', 'Moncloa', 'Moratalaz', 'Puente de Vallecas',
       'Retiro', 'Salamanca', 'San Blas', 'Tetuán', 'Usera', 'Vicálvaro',
       'Villa de Vallecas', 'Villaverde', 'Arganzuela', 'ciudadlineal',
       'puentedevallecas', 'sanblas', 'villadevallecas', 'Madrid',
       'CiudadLineal', 'PuentedeVallecas', 'SanBlas', 'VilladeVallecas']

lower = ['barajas', 'carabanchel', 'centro', 'chamartín', 'chamberí',
       'ciudad lineal', 'fuencarral', 'hortaleza', 'latina', 'moncloa',
       'moratalaz', 'puente de vallecas', 'retiro', 'salamanca',
       'san blas', 'tetuán', 'usera', 'vicálvaro', 'villa de vallecas',
       'villaverde', 'arganzuela', 'ciudad lineal', 'puente de vallecas', 
       'san blas', 'villa de vallecas', 'madrid', 'ciudad lineal', 'puente de vallecas', 
       'san blas', 'villa de vallecas']

districts_dict = dict(zip(upper, lower))

def clean_tabula_read_pdf_parsed_data(data_frame, year, columns, district=0):
    # extract the total of madrid data
    all_madrid = data_frame.tail(1)
    if len([x for x in all_madrid.iloc[0,district].split() if x in ['Madrid','madrid']]) > 0:
        data_frame.drop(data_frame.index[len(data_frame)-1], inplace=True)
    
    # include header as a new row
    a = pd.DataFrame(data_frame.columns.tolist()).T
    a.columns = data_frame.columns.tolist()
    data_frame_ = pd.concat([data_frame, a], axis=0)
    
    # clean district names
    if district == 0:
        districts = data_frame_[data_frame.columns.tolist()[district]].str.split().apply(lambda x: ' '.join(x[1:]))
        data_frame_[data_frame.columns.tolist()[district]] = districts
    else:
        districts = data_frame_.iloc[:,district].tolist()
        data_frame_[data_frame.columns.tolist()[district]] = districts
    
    # add all madrid data, rename columns and add year label
    if len([x for x in all_madrid.iloc[0,district].split() if x in ['Madrid','madrid']]) > 0:
        c = pd.concat([data_frame_, all_madrid], axis=0)
    else:
        c = data_frame_.copy()
    c.columns = columns
    c['year'] = year
    return c.loc[:,['district','price_m2','year']]

def clean_rent_prices_2012(data):
    a = data.loc[3:]
    a.loc[:,'price_m2'] = a.loc[:,0].str.split().apply(lambda x: x[-1:][0])
    a.loc[:,'jun_prev_year'] = a.loc[:,0].str.split().apply(lambda x: x[-2:-1][0])
    a.loc[:,'district'] = a.loc[:,0].str.split().apply(lambda x: ' '.join(x[1:-2]))
    a.loc[:,'jun_year'] = a[1]
    a.loc[:,'year'] = 2012
    return a.loc[:,['district','price_m2','year']]

def clean_sale_prices_special(data):
    data_ = data.copy()
    col_name = data_.columns[2].split()[1]
    data_[col_name] = data_.iloc[:,2].str.split().apply(lambda x: x[1])
    return data_

### 1. Build AirBnB dataset 2010-2017

In [3]:
# Load and join AirBnB listings and reviews
airbnb_source = "http://insideairbnb.com/get-the-data.html"
abnb_listings = pd.read_csv("./data/airbnb_listints.csv")
abnb_reviews = pd.read_csv("./data/airbnb_reviews.csv").rename(columns={'date':'date_review'})

# Assign reviews to each listing
abnb = abnb_listings.merge(abnb_reviews, left_on='id', right_on='listing_id')

abnb.to_csv('./data/airbnb_complete.csv', index=False, encoding='utf-8')

### 2. Load mortage executions historical data for madrid (CGPJ) 2007-2017

In [3]:
# Load CGPJ evictions data
cgpj_source = "http://www.poderjudicial.es/cgpj/es/Temas/Estadistica-Judicial/Estudios-e-Informes/Efecto-de-la-Crisis-en-los-organos-judiciales/"
mort_executions = pd.read_csv("./data/madrid_ejec_hipotec_cgpj.csv")

# Clean and transpose source data
mort_executions = mort_executions.ix[:,['Total 2007', 'Total 2008', 'Total 2009', 'Total 2010', 'Total 2011',
                   'Total 2012', 'Total 2013', 'Total 2014', 'Total 2015', 'Total 2016',
                   'Total 2017']].T.reset_index().rename(columns={0:'mortage_executions','index':'year'})
mort_executions['year'] = mort_executions['year'].apply(lambda x: x.split(' ',1)[1])

mort_executions.to_csv("./data/c_madrid_ejec_hipotec_cgpj.csv", index=False, encoding='utf-8')

### 3. Load data of price/m2 for selling 2007-2017

In [5]:
# load readable
sale_2014 = read_pdf('./data/anio-2014.pdf')
sale_2015 = read_pdf('./data/anio-2015.pdf')
sale_2016 = read_pdf('./data/anio-2016.pdf')
sale_2017 = read_pdf('./data/anio-2017.pdf')

In [6]:
# parse readable data
cols_2 = ['district','maximum','jun_prev_year','trash','jun_year','max_var','anual_var','quarterly_var','price_m2']
cols_3 = ['trash','district','maximum','jun_prev_year','price_m2','jun_year','max_var','anual_var','quarterly_var']

sale_2014_ = clean_sale_prices_special(sale_2014)
sale_2016_ = clean_sale_prices_special(sale_2016)
sale_2015.iloc[:,1] = sale_2015.iloc[:,1].str.replace(' ', '')
sale_2017.iloc[:,1] = sale_2017.iloc[:,1].str.replace(' ', '')

sale_2014_ = clean_tabula_read_pdf_parsed_data(sale_2014_, 2014, cols_2)
sale_2015_ = clean_tabula_read_pdf_parsed_data(sale_2015, 2015, cols_3, district=1)
sale_2016_ = clean_tabula_read_pdf_parsed_data(sale_2016_, 2016, cols_2)
sale_2017_ = clean_tabula_read_pdf_parsed_data(sale_2017, 2017, cols_3, district=1)

sale_readable = pd.concat([sale_2014_, sale_2015_, sale_2016_, sale_2017_], axis=0)
sale_readable["district"].replace(districts_dict, inplace=True)

In [7]:
# load non readable manually parsed
sale_non_readable = pd.read_csv('./data/sale_manually_parsed.csv')

# concatenate all sale data
sale_dataset = pd.concat([sale_readable, sale_non_readable], axis=0)

In [8]:
sale_dataset.to_csv('./data/sale_price_m2_districts.csv', index=False, encoding='utf-8')

### 4. Load data of price/m2 for renting 2007-2017

In [9]:
# load readable
rent_2009 = read_pdf('./data/alquiler-2009.pdf')
rent_2010 = read_pdf('./data/alquiler-2010.pdf')
rent_2011 = read_pdf('./data/alquiler-2011.pdf')
rent_2012 = read_pdf('./data/alquiler-2012.pdf', multiple_tables=True)[0]
rent_2014 = read_pdf('./data/alquiler-2014.pdf')
rent_2015 = read_pdf('./data/alquiler-2015.pdf')
rent_2016 = read_pdf('./data/alquiler-2016.pdf')
rent_2017 = read_pdf('./data/alquiler-2017.pdf')

In [10]:
# parse and clean readable data
cols_2 = ['district','jan_year','price_m2','ene_next_year','trash']
cols_12 = ['district','jun_prev_year','price_m2','jun_year']
cols_1 = ['district','jun_prev_year','price_m2','jun_year','max_var','anual_var','quarterly_var']
cols_3 = ['district','maximum','jun_prev_year','price_m2','jun_year','max_var','anual_var','quarterly_var']

rent_2009_ = clean_tabula_read_pdf_parsed_data(rent_2009, 2009, cols_1)
rent_2010_ = clean_tabula_read_pdf_parsed_data(rent_2010, 2010, cols_2)
rent_2011_ = clean_tabula_read_pdf_parsed_data(rent_2011, 2011, cols_2)
rent_2012_ = clean_rent_prices_2012(rent_2012)
rent_2014_ = clean_tabula_read_pdf_parsed_data(rent_2014, 2014, cols_3)
rent_2015_ = clean_tabula_read_pdf_parsed_data(rent_2015, 2015, cols_3)
rent_2016_ = clean_tabula_read_pdf_parsed_data(rent_2016, 2016, cols_3)
rent_2017_ = clean_tabula_read_pdf_parsed_data(rent_2017, 2017, cols_3)

rent_readable_dataset = pd.concat([rent_2009_,rent_2010_,rent_2011_,rent_2012_,rent_2014_,
                                   rent_2015_,rent_2016_,rent_2017_], axis=0).reset_index(drop=True)
rent_readable_dataset["district"].replace(districts_dict, inplace=True)

In [11]:
# load non readable manually parsed
rent_non_readable = pd.read_csv('./data/rent_manually_parsed.csv')

# concatenate with previous dataset
rent_dataset = pd.concat([rent_readable_dataset, rent_non_readable], axis=0)

In [12]:
rent_dataset.to_csv('./data/rent_price_m2_districts.csv', index=False, encoding='utf-8')